In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [7]:
# Buckwalter → Arabic Unicode map
buckwalter_map = {
    "'": "ء",  "|": "آ",  ">": "أ",  "&": "ؤ",  "<": "إ",  "}": "ئ",
    "A": "ا",  "b": "ب",  "p": "ة",  "t": "ت",  "v": "ث",  "j": "ج",
    "H": "ح",  "x": "خ",  "d": "د",  "*": "ذ",  "r": "ر",  "z": "ز",
    "s": "س",  "$": "ش",  "S": "ص",  "D": "ض",  "T": "ط",  "Z": "ظ",
    "E": "ع",  "g": "غ",  "_": "ـ",  "f": "ف",  "q": "ق",  "k": "ك",
    "l": "ل",  "m": "م",  "n": "ن",  "h": "ه",  "w": "و",  "Y": "ى",
    "y": "ي",  "F": "ً",  "N": "ٌ",  "K": "ٍ",  "a": "َ",  "u": "ُ",
    "i": "ِ",  "~": "ّ",  "o": "ْ",  "`": "ٰ",  "{": "ٱ"
}

def buckwalter_to_arabic(text):
    return "".join(buckwalter_map.get(ch, ch) for ch in text)


# preparing tha dataset

all of the datasets have samiliar distribution as the records are 5-20 seconds and for each split 200 records which satfies the minimum amount of data

In [2]:
from datasets import load_dataset, concatenate_datasets, Audio

# 1️⃣ Load the raw “train” splits
egy_raw = load_dataset("MightyStudent/Egyptian-ASR-MGB-3", split="train")
cl_raw  = load_dataset("MBZUAI/ClArTTS",           split="train")
msa_raw = load_dataset("halabi2016/arabic_speech_corpus", split="train")

# 2️⃣ Process Egyptian‑Arabic: rename ‘sentence’→‘text’, drop all else
egy = egy_raw.rename_column("sentence", "text")
cols_to_keep = {"audio","text"}
egy = egy.remove_columns([c for c in egy.column_names if c not in cols_to_keep])

# 3️⃣ Process MSA: drop all except audio & text
msa = msa_raw.remove_columns([c for c in msa_raw.column_names if c not in cols_to_keep])

# 4️⃣ Classical‑Arabic: first subsample 200, then wrap only those
cl200 = cl_raw.shuffle(seed=0).select(range(200))

def wrap_classic(ex):
    return {
      "text":  ex["text"],
      "audio": {"array": ex["audio"], "sampling_rate": ex["sampling_rate"]}
    }

cl200 = cl200.map(
    wrap_classic,
    remove_columns=[c for c in cl200.column_names if c not in cols_to_keep]
)

# 5️⃣ Cast each of the three to Audio(sampling_rate=16000)
egy200 = egy.shuffle(seed=0).select(range(200)).cast_column("audio", Audio(sampling_rate=16000))
cl200 = cl200.cast_column("audio", Audio(sampling_rate=16000))
msa200 = msa.shuffle(seed=0).select(range(200)).cast_column("audio", Audio(sampling_rate=16000))

# 6️⃣ Concatenate → 600 examples, uniform schema
combined = concatenate_datasets([egy200, cl200, msa200])
print(combined)


README.md:   0%|          | 0.00/3.27k [00:00<?, ?B/s]

(…)-00000-of-00002-69f2a5d14e629dff.parquet:   0%|          | 0.00/467M [00:00<?, ?B/s]

(…)-00001-of-00002-860b0ca7a1fb4dc3.parquet:   0%|          | 0.00/454M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1159 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

...

The repository for halabi2016/arabic_speech_corpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/halabi2016/arabic_speech_corpus.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] غ
The repository for halabi2016/arabic_speech_corpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/halabi2016/arabic_speech_corpus.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/1813 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'text'],
    num_rows: 600
})


In [11]:
# Assuming combined is your Dataset and buckwalter_to_arabic is defined

def convert_text(example, idx):
    if 400 <= idx < 600:
        example['text'] = buckwalter_to_arabic(example['text'])
    return example

# Apply the conversion to the specific range
combined = combined.map(convert_text, with_indices=True)

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [21]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.1 MB/s eta 0:00:00


## wav2vec2-large-xlsr-53-arabic model

In [22]:
from transformers import pipeline
from jiwer import wer
import librosa

# Initialize the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="jonatasgrosman/wav2vec2-large-xlsr-53-arabic")


config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

Device set to use cuda:0


In [25]:
def resample_audio(audio_array, orig_sr, target_sr=16000):
    if orig_sr != target_sr:
        return librosa.resample(audio_array, orig_sr=orig_sr, target_sr=target_sr)
    return audio_array

# Extract and resample audio arrays
audios = [
    resample_audio(sample["audio"]["array"], sample["audio"]["sampling_rate"])
    for sample in combined
]

# Transcribe the audio samples in batches
transcriptions = pipe(audios, batch_size=16)
predicted_texts = [transcription["text"] for transcription in transcriptions]

# Extract ground truth texts
ground_truth_texts = combined["text"]

# Calculate Word Error Rate (WER)
error_rate = wer(ground_truth_texts, predicted_texts)
print(f"Word Error Rate: {error_rate}")

Word Error Rate: 0.7859896791031497
